In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yfin
import seaborn as sns
import math
from numpy import linalg as LA

In [2]:
# Download stock prices from Yahoo Finance and set the time period for download
start = datetime.date(2018, 1, 2)
end = datetime.date(2023, 12, 31)
stocks = yfin.download(["AAPL", "F", "WMT","PFE"], start, end, auto_adjust = False)["Adj Close"]
stocks.head()

[*********************100%***********************]  4 of 4 completed


Ticker,AAPL,F,PFE,WMT
Date,,,,
2018-01-02,40.426834,8.470929,25.081699,29.041031
2018-01-03,40.419788,8.537838,25.267540,29.294361
2018-01-04,40.607540,8.685043,25.322605,29.320866
2018-01-05,41.069855,8.832247,25.370787,29.494659
2018-01-08,40.917324,8.798794,25.088587,29.930611


In [3]:
stocks.index = pd.to_datetime(stocks.index).strftime("%Y-%m-%d")

In [4]:
# In order to calculate the returns of the stocks, we need to drop the NA rows.
stocks_returns = stocks[["AAPL", "F", "WMT","PFE"]].dropna().pct_change()
stocks_returns = stocks_returns.dropna()
stocks_returns.head()

Ticker,AAPL,F,WMT,PFE
Date,,,,
2018-01-03,-0.000174,0.007899,0.008723,0.007409
2018-01-04,0.004645,0.017242,0.000905,0.002179
2018-01-05,0.011385,0.016949,0.005927,0.001903
2018-01-08,-0.003714,-0.003788,0.014781,-0.011123
2018-01-09,-0.000115,-0.005323,-0.012007,-0.001098


In [5]:
# Perform SVD for stock returns
U, s, VT = np.linalg.svd(stocks_returns)

In [6]:
# Present the result
print("Stock Returns Matrix Dimension:")
print(stocks_returns.shape)
print("\nDimension of Matrix U:")
print(U.shape)
print("\nSingular values:")
print(s)
print("\nDimension of Matrix V^T:")
print(VT.shape)

Stock Returns Matrix Dimension:
(1508, 4)

Dimension of Matrix U:
(1508, 1508)

Singular values:
[1.11696763 0.72489541 0.55820555 0.47159232]

Dimension of Matrix V^T:
(4, 4)


In [7]:
# Standardize stock returns dataset
stocks_returns_means = stocks_returns.mean()
stocks_returns_stds = stocks_returns.std()
standardized_returns = (stocks_returns - stocks_returns_means) / stocks_returns_stds
standardized_returns.head()

Ticker,AAPL,F,WMT,PFE
Date,,,,
2018-01-03,-0.070363,0.286261,0.584156,0.446786
2018-01-04,0.171151,0.647643,0.029987,0.124448
2018-01-05,0.508912,0.636331,0.385985,0.107403
2018-01-08,-0.247748,-0.165762,1.013511,-0.695387
2018-01-09,-0.067380,-0.225168,-0.885170,-0.077508


In [8]:
# Calculate covariance for standardized return matrix
standardized_returns_dvd_sqrt_n=(standardized_returns/math.sqrt(len(standardized_returns)-1))
standardized_returns_cov = standardized_returns_dvd_sqrt_n.T@standardized_returns_dvd_sqrt_n
standardized_returns_cov

Ticker,AAPL,F,WMT,PFE
Ticker,,,,
AAPL,1.000000,0.376461,0.365246,0.324438
F,0.376461,1.000000,0.185249,0.224384
WMT,0.365246,0.185249,1.000000,0.296375
PFE,0.324438,0.224384,0.296375,1.000000


In [9]:
# Use SVD to calculate eigenvectors and eigenvalues of the covariance matrix of standardized returns
U_st_return, s_st_return, VT_st_return = np.linalg.svd(standardized_returns_dvd_sqrt_n)
print("\nSquared Singular values (eigenvalues):")
print(s_st_return**2)
print("\nMatrix V (eigenvectors)")
print(VT_st_return.T)


Squared Singular values (eigenvalues):
[1.89474981 0.83555303 0.7106495  0.55904767]

Matrix V (eigenvectors)
[[ 0.56634774  0.14011625 -0.21637434 -0.78281531]
 [ 0.4596478   0.75759289  0.01833619  0.46307742]
 [ 0.48586776 -0.5322672  -0.55857219  0.41063519]
 [ 0.48156687 -0.35087331  0.80052673  0.06432911]]


In [10]:
eigenvalues, eigenvectors = LA.eig(standardized_returns_cov)
idx = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]
eigenvalues

array([1.89474981, 0.83555303, 0.7106495 , 0.55904767])

In [11]:
eigenvectors

array([[-0.56634774,  0.14011625, -0.21637434, -0.78281531],
       [-0.4596478 ,  0.75759289,  0.01833619,  0.46307742],
       [-0.48586776, -0.5322672 , -0.55857219,  0.41063519],
       [-0.48156687, -0.35087331,  0.80052673,  0.06432911]])